In [15]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# pd.set_option('display.max_colwidth', None)

In [68]:
#пропиши путь к файлу
data = pd.read_csv('../datasets/netflix_titles.csv')

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [ ]:
data.head()

# Task 1

Сколько в выборке сериалов?

In [7]:
data['type'].value_counts()

Movie      6131
TV Show    2676
Name: type, dtype: int64

In [17]:
print("В выборке", data['type'].value_counts()[1], "сериалов.")

В выборке 2676 сериалов.


In [27]:
cnt = (data['type'].str.contains('TV Show').sum())
print(f'В выборке находится {cnt} сериалов.')

В выборке находится 2676 сериалов


# Task 2

Используя параметр 'duration' вычисли, сколько сезонов в среднем длится один сериал?

Замени пропуски в 'duration' на 1 - хотя бы один уже выпустили :)

Округли ответ до целых в большую сторону.

In [117]:
ser = data[data['type'] == 'TV Show'] # создаем отдельный df для сериалов
ser['duration'].isna().sum() #проверяем нет ли пустых значений

pandas.core.frame.DataFrame

In [103]:
ser.duration = ser.duration.str.replace(' Seasons', '')
ser.duration = ser.duration.str.replace(' Season', '')

In [105]:
ser.duration = ser.duration.astype('int')

In [114]:
avg_dur = ser.duration.mean()
print(f"Один сериал в среднем длится {round(avg_dur, 2)} сезонов.")

Один сериал в среднем длится 1.76 сезонов.


# Task 3

Собери словарь с классификацией рейтингов на русском языке и замени значения в столбце data['category'], используя метод map(). 
Для разметки можно использовать [информацию с этой странички](https://www.primevideo.com/help?nodeId=GFGQU3WYEG6FSJFJ).

В колонку с рейтингом могли попасть ошибки. Замени в этих строках и в строках с пропусками рейтинг на 'Нет данных'.

В какой категории рейтинга больше всего фильмов?

In [39]:
raiting = pd.read_csv('AgeRaiting_dict.csv')

In [44]:
keys = list(raiting['Key'])
values = list(raiting['Value'])

In [46]:
raiting_dict = dict(zip(keys,values ))

In [47]:
print(raiting_dict)

{'G': '0+', 'NC-17': '18+', 'NR': '18+', 'PG': '7+', 'PG-13': '13+', 'R': '18+', 'TV-14': '16+', 'TV-G': '0+', 'TV-MA': '18+', 'TV-PG': '7+', 'TV-Y': '0+', 'TV-Y7': '7+', 'TV-Y7-FV': '7+', 'UR': 'Unrated'}


In [78]:
data = pd.read_csv('../datasets/netflix_titles.csv')
print(data['rating'].value_counts().sum())
print(data['rating'].value_counts())

8803
TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
TV-Y7-FV       6
NC-17          3
UR             3
74 min         1
84 min         1
66 min         1
Name: rating, dtype: int64


In [79]:
data = pd.read_csv('../datasets/netflix_titles.csv')
data['rating'] = data['rating'].map(raiting_dict).fillna('Unrated')
print(data['rating'].value_counts().sum())
print(data['rating'].value_counts())

8807
18+        4089
16+        2160
7+         1490
0+          568
13+         490
Unrated      10
Name: rating, dtype: int64


# Task 4

Сколько разных жанров на платформе? Выведи уникальные значения из поля 'listed_in'

In [137]:
data.listed_in.nunique()

514

# Task 5

Какой жанр произведений чаще всего выпускали в США за последние 5 лет с 2017 по 2021? 

Так как Netflix - американский сервис, то все пропуски в поле 'страна' можно заменить на United States и не исключать их из статистики

In [139]:
data.country.fillna('United States', inplace = True)

In [164]:
years = [2017,2018, 2019, 2020, 2021]
countries = ['United States']

In [166]:
query_1 = data.query("release_year in @years & country in @countries")

In [169]:
query_1['listed_in'].value_counts().idxmax()

'Documentaries'

# Дополнительное задание *

Выведи топ 3 самых популярных актера в жанре Comedies

In [85]:
comedies = data[(data.listed_in.str.contains('Comedies')) &
               (data.cast.isna() == False)][['cast']].copy()

In [84]:
comedies.head()

,cast
4,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K..."
9,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T..."
13,"Klara Castanho, Lucca Picon, Júlia Gomes, Marc..."
15,"Logan Browning, Brandon P. Bell, DeRon Horton,..."
22,"Kamal Hassan, Meena, Gemini Ganesan, Heera Raj..."


In [90]:
all_actors = comedies['cast'].str.split(',').explode()
actor_counts = all_actors.value_counts()
top_actors = actor_counts.head(3)

In [91]:
print(top_actors)

 Anupam Kher     20
Adam Sandler     18
 Paresh Rawal    16
Name: cast, dtype: int64


In [93]:
actors_count = {}
for row in comedies['cast']:
    actors = row.split(',')
    for actor in actors:
        actors_count[actor] = actors_count.get(actor, 0) + 1
    

**actor_counts.get(actor, 0)**:    
эта строка извлекает текущее количество актеров из словаря actor_counts . 
Метод get() используется для доступа к значению, связанному с actor ключом. Если ключ ( actor) присутствует в словаре, он возвращает соответствующее значение (текущее количество). Если ключ не найден в словаре, он возвращает указанное значение по умолчанию (0 в данном случае).

**+ 1**: Эта часть увеличивает полученный счетчик на 1. Он обновляет счетчик, чтобы отразить вхождение актера в текущую строку.

**actor_counts[actor]**: Наконец, эта строка присваивает обновленный счетчик ключу actorв actor_countsсловаре. Если актор уже присутствует в качестве ключа в словаре, значение счетчика будет обновлено. Если актер еще не присутствует, в словарь будет добавлена ​​новая пара ключ-значение, представляющая актера и его количество.

Таким образом, эта строка кода извлекает текущее количество актеров из словаря, увеличивает его на 1, и обновляет количество в словаре. Он эффективно подсчитывает количество вхождений каждого актера в кадре данных, обновляя словарь с актером в качестве ключа и их соответствующим счетчиком в качестве значения.








In [94]:
sorted_actors = sorted(actors_count.items(), key = lambda x: x[1], reverse = True)

**actor_counts.items()**: эта часть получает список пар ключ-значение из actor_counts словаря. Каждая пара представляет актера и его количество.

**key=lambda x: x[1]**: эта часть определяет ключевую функцию с помощью лямбда-выражения. Он указывает, что сортировка должна основываться на втором элементе ( x[1]) каждой пары ключ-значение. В данном случае это означает сортировку по количеству каждого актера.

**reverse=True**: этот параметр используется для указания сортировки по убыванию. Актеры с более высокими значениями будут отображаться первыми в отсортированном списке.

**sorted()**: эта функция берет список пар ключ-значение и выполняет сортировку на основе указанной ключевой функции. Он возвращает новый список с парами, отсортированными в нужном порядке.

**sorted_actors**: Наконец, эта строка присваивает отсортированный список пар ключ-значение переменной sorted_actors.

Таким образом, эта строка кода сортирует пары ключ-значение из словаря actor_countsна основе количества (второго элемента) каждой пары в порядке убывания. Он возвращает новый список ( sorted_actors) с парами, отсортированными в соответствии с указанными критериями.

In [103]:
for i, (actor, count) in enumerate(sorted_actors):
    print(f'{actor}: {count}')
    if i == 2:  # Prints only the top three actors
        break

 Anupam Kher: 20
Adam Sandler: 18
 Paresh Rawal: 16
